# Programming Assignment 5 [Neural Networks]

Class: INF552

Student: Jiashi Chen

USCID: 4684194123

### Import necessary module

In [1]:
import pandas as pd
import numpy as np
import random

### Obtain training dateset

In [2]:
data_train = pd.read_csv("downgesture_train.list", header = None)
data_train.columns = ['Scr']
data_train["Label"] = data_train["Scr"].map(lambda x: 1 if x.split("_")[1] == "down" else 0) #Assign label to each data set according to its file name
data_train

,Scr,Label
0,gestures/A/A_down_4.pgm,1
1,gestures/A/A_down_6.pgm,1
2,gestures/A/A_down_7.pgm,1
3,gestures/A/A_down_8.pgm,1
4,gestures/A/A_down_9.pgm,1
...,...,...
179,gestures/M/M_stop_9.pgm,0
180,gestures/M/M_up_1.pgm,0
181,gestures/M/M_up_2.pgm,0
182,gestures/M/M_up_3.pgm,0


### Load image based on its pgm format

In [3]:
def load_pgm_image(pgm):
    with open(pgm, 'rb') as f:
        f.readline()   
        f.readline()   
        xs, ys = f.readline().split()  
        xs = int(xs)
        ys = int(ys)
        max_scale = int(f.readline().strip())
        image = []
        for _ in range(xs * ys):
            image.append(f.read(1)[0] / max_scale)
        
        image.insert(0,1.0) #add bias
        return np.mat(image)

In [4]:
load_pgm_image("gestures/B/B_up_1.pgm")

matrix([[1.        , 0.79607843, 0.80784314, 0.81568627, 0.82745098,
         0.83921569, 0.84313725, 0.84705882, 0.85882353, 0.85882353,
         0.85490196, 0.85490196, 0.85098039, 0.84705882, 0.84705882,
         0.84705882, 0.84313725, 0.84313725, 0.83921569, 0.83529412,
         0.83529412, 0.82745098, 0.82352941, 0.81568627, 0.81176471,
         0.80392157, 0.79607843, 0.78431373, 0.77647059, 0.76862745,
         0.75686275, 0.74117647, 0.72941176, 0.8       , 0.81176471,
         0.81960784, 0.83529412, 0.84705882, 0.85490196, 0.85882353,
         0.8745098 , 0.87058824, 0.8627451 , 0.8627451 , 0.85882353,
         0.85490196, 0.85098039, 0.85098039, 0.84705882, 0.84313725,
         0.84313725, 0.83921569, 0.83529412, 0.83137255, 0.82352941,
         0.81960784, 0.81568627, 0.80784314, 0.8       , 0.78823529,
         0.78039216, 0.77254902, 0.75686275, 0.74509804, 0.73333333,
         0.80784314, 0.81568627, 0.82745098, 0.84313725, 0.86666667,
         0.87843137, 0.88235294, 0

### Sigmoid function

In [5]:
def sigmoid(z):
    return (1/(1 + np.exp(-z)))

### Error function

In [6]:
def error(x, label):
    return (x - label)**2

### Initialize weight for Neural Network

In [7]:
def initial_weight(number_input, number_hidden, number_output):
    w1 = np.random.randint(-100, 100,(number_input,number_hidden))/10000
    w2 = np.random.randint(-100, 100,(number_hidden+1,number_output))/10000
    return np.mat(w1), np.mat(w2)

### Compute all x in forward direction

In [8]:
def forward_propagation(img, w1, w2):
    s1 = img * w1 #Calculate the hidden layer
    x1 = sigmoid(s1)
    x1 = np.insert(x1, 0, values=1.0, axis=1)  #add bias
    s2 = x1 * w2
    x2 = sigmoid(s2)
    return x1, x2[0, 0]

### Update w based on backward propagation

In [9]:
def backward_propagation(img, x1, x2, label, w1, w2): ##sigmoid
    base_delta = 2 * (x2 - label) * (x2 - x2**2)
    delta1 = np.multiply((x1 - np.multiply(x1, x1)), w2.T * base_delta)
    w1 -= 0.1 * img.T * delta1[0, 1:]
    w2 -= 0.1 * base_delta * x1.T
    return w1, w2

In [10]:
# def backward_propagation(img, x1, x2, label, w1, w2):  ###tanh
#     base_delta = 2 * (x2 - label) * (1 - x2**2)
#     delta1 = np.multiply((np.mat([1] * 101) - np.multiply(x1, x1)), w2.T * base_delta)
# #     print(delta1)
#     w1 -= 0.1 * img.T * delta1[0, 1:]
#     w2 -= 0.1 * base_delta * x1.T
#     return w1, w2

### Fit the model

In [11]:
def Neural_Networks_BGD(data, number_hidden, number_output):    #batch gradient descent
    number_input = load_pgm_image(data.iloc[0,0]).shape[1]
    w1, w2 = initial_weight(number_input, number_hidden, number_output)
    count = 0
    while(count < 1000):
        count += 1
        print(count)
        for index, row in data.iterrows():
            path = row["Scr"]
            label = row["Label"]
            im = load_pgm_image(path)
            x1, x2 = forward_propagation(im, w1, w2)
            w1, w2 = backward_propagation(im, x1, x2, label, w1, w2)
    return w1, w2

In [12]:
def Neural_Networks_Stochastic(data, number_hidden, number_output):   #Stochastic gradient descent
    number_input = load_pgm_image(data.iloc[0,0]).shape[1]
    m = data.shape[0]
    w1, w2 = initial_weight(number_input, number_hidden, number_output)
    count = 0
    while(count < 1000):
        count += 1
        temp_num = random.randint(1, m)
        while (temp_num > 0):
            temp_num -= 1
            temp_index = random.randint(0, m-1)
            path = data.iloc[temp_index, 0]
            label = data.iloc[temp_index, 1]
            im = load_pgm_image(path)
            x1, x2 = forward_propagation(im, w1, w2)
            w1, w2 = backward_propagation(im, x1, x2, label, w1, w2)
        
    return w1, w2

### Classify the output

In [13]:
def classify(x2):
    return 1 if x2 >= 0.5 else 0

### Compute the accuracy

In [14]:
def accuracy(data, w1, w2):
    right = 0
    origin_label = []
    predict_label = []
    for index, row in data.iterrows():
        path = row["Scr"]
        label = row["Label"]
        im = load_pgm_image(path)
        x1, x2 = forward_propagation(im, w1, w2)
        origin_label.append(label)
        predict_label.append(classify(x2))
        if (classify(x2) == label):
            right += 1
    return right/data.shape[0], origin_label, predict_label

In [15]:
# w1, w2 = Neural_Networks(data)
# predict(data, w1, w2)

In [25]:
w1, w2 = Neural_Networks_Stochastic(data_train, 100, 1)
accuracy_train, origin_label, predict_label = accuracy(data_train, w1, w2)

In [26]:
accuracy_train

1.0

### Obtain test dateset

In [18]:
data_test = pd.read_csv("downgesture_test.list", header = None)
data_test.columns = ['Scr']
data_test["Label"] = data_test["Scr"].map(lambda x: 1 if x.split("_")[1] == "down" else 0) #Assign label to each data set according to its file name
data_test

,Scr,Label
0,gestures/A/A_down_1.pgm,1
1,gestures/A/A_down_2.pgm,1
2,gestures/A/A_hold_1.pgm,0
3,gestures/A/A_hold_10.pgm,0
4,gestures/A/A_stop_1.pgm,0
...,...,...
78,gestures/K/K_hold_3.pgm,0
79,gestures/K/K_stop_1.pgm,0
80,gestures/K/K_stop_2.pgm,0
81,gestures/K/K_stop_1.pgm,0


### Compute the accuracy of prediction of the labels for the gestures in the test images

In [27]:
accuracy_test, origin_label_test, predict_label_test = accuracy(data_test, w1, w2)

In [28]:
accuracy_test

0.891566265060241

In [30]:
predict_label_test

[1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# Part 2: Software Familiarization 

### Import necessary module

In [22]:
from sklearn.neural_network import MLPClassifier

### Fit the model to data matrix X and target(s) y.

In [23]:
X = [(load_pgm_image(img)[0,1:]).tolist()[0] for img in data_train["Scr"]]
y = data_train["Label"].tolist()
mlp = MLPClassifier(solver='sgd', alpha=0,
                  hidden_layer_sizes=(100,), activation='logistic', learning_rate_init=0.1,
                  max_iter=1000)
mlp.fit(X, y)

MLPClassifier(activation='logistic', alpha=0, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.1, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='sgd',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [24]:
X_test = [(load_pgm_image(img)[0,1:]).tolist()[0] for img in data_test["Scr"]]
mlp.predict(X_test)

array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])